<a href="https://colab.research.google.com/github/Strix9289/Introduction-of-Deep-Learning/blob/master/Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最適化アルゴリズム

+ #### 最適化アルゴリズム

    - #### 勾配降下法

    - #### 確率的勾配降下法(SGD)

    - #### Adagrad

    - #### RMSprop

    - #### NAG

    - #### AdaDelta

    - #### Adam



## **勾配降下法**
## **確率的勾配降下法SGD**

```py
keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
```

引数は、

* lr: 学習率、0以上の実数
* momentum: モーメンタム、0以上の実数（前回のパラメータ更新量を反映させる比率（3.2.2で説明））
* decay: 更新毎の学習率の減衰率、0以上の実数
* nesterov: Nesterov momentumを適用するかどうか（Trueならモーメンタム項の計算を1ステップ先読みして評価します）

いわゆる基本中の基本
$$
\boldsymbol{w}_{t+1}\leftarrow\boldsymbol{w}_t-\eta\Delta\boldsymbol{w}_t
$$
 $\boldsymbol{w}$:パラメータ , $\eta$:学習率
この場合、$\eta$が大きいと最適値を通り越し、小さいとなかなか収束しないという事態に陥る


SGD
前ステップの更新を加味することで、 勾配の変化を滑らかにします。

これによって、「通り過ぎる」ことによる行ったり来たりの振動を抑制することができます。
$$
\boldsymbol{m}_t\leftarrow\beta_1\boldsymbol{m}_{t-1}+\eta\Delta\boldsymbol{w}_t
$$

$$
\boldsymbol{w}_{t+1}\leftarrow\boldsymbol{w}_t-\boldsymbol{m}_t
$$
$\boldsymbol{m}$:前回の更新量, $\beta$:モーメンタム



また、勾配変化の少ない（パラメータ空間内の）通常の斜面においては、

他の勾配変化の大きい斜面と比較して学習率が上昇し、加速的に学習が進むという効果を持っています。

Momentumでは前ステップの更新量を加味する割合を'momentum'として指定します。

なお、`momentum=0`では通常のSGDと一致することもあり、KerasではSGDクラスのmomentum引数を調整することで使用できます。

通常は`momentum=0.9`程度に設定します。

メリットは
- 学習の時間が早い
- 広範囲を探索できるために局所的な解に陥りにくい

ということで画像分類の問題では2020年4月現在でもよくつかわれている



## Adagrad
全体の学習率を書く方向度とに過去の勾配の累積で割りびうことで、勾配が大きかった方向の学習率を下げ、小さかった方向の学習率を上げる
AdaGradは学習の初期に勾配が大きいとすぐに更新量が小さくなってしまい、学習がストップしてしまうという欠点があるため、
学習率の選択、また重みの初期値の選択は慎重に行う必要があるという欠点をもっていることは気にしておくとよいでしょう。

通常`lr=0.01`
```py
from keras.optimizers import Adagrad
model.compile(loss='categorical_crossentropy', optimizer=Adagrad(lr=0.01, epsilon=1e-08, decay=0.0))
```

更新式は次の通りです。（$\Delta\boldsymbol{w}^{(t)}_i$：$t$回目のパラメータ更新量＠第$i$成分、$\nabla\mathrm{E}(\boldsymbol{w}^{(t)})_i$：$t$回目のパラメータによる勾配＠第$i$成分、$\eta$：学習率（引数lr））

$$
    \Delta\boldsymbol{w}^{(t)}_i = -\frac{\eta}{\sqrt{\sum^{t}_{s=1}\left(\nabla\mathrm{E}(\boldsymbol{w}^{(s)})_i\right)^2 + \varepsilon}}\nabla\mathrm{E}(\boldsymbol{w}^{(t)})_i
$$

なお、$\varepsilon$は計算機上での0割りを回避するためのもので、ごく小さい値（`epsilon=10^-8`）を指定します。


## RMSprop

勾配の情報が指数的な減衰によって次第に忘却されるように更新式を変更する

ふつうは`rho=0.9`  rhoはどれだけ過去の勾配を重視するかを表す。
```py
# RMSPropの実装例
from keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0))
```

勾配の情報が出にくいパラメータは一気に大きく更新し、勾配の情報が出やすいパラメタは細かく少しずつ更新する。
例えば、自然言語ではめったに出現しない単語に関しては、一気に更新したほうが正確な値まで一気に更新できる。

（$\Delta\boldsymbol{w}^{(t)}_i$、$\nabla\mathrm{E}(\boldsymbol{w}^{(t)})_i$、$\eta$：AdaGradと同じ、$\rho$：勾配情報の減衰率（引数rho））

$$
    v_i^{(t)} = \rho v_i^{(t-1)} + (1-\rho)(\nabla\mathrm{E}(\boldsymbol{w}^{(t)})_i)^2 \quad (v_i^{(0)}=0)\\
    \Delta\boldsymbol{w}^{(t)}_i=-\frac{\eta}{\sqrt{v_i^{(t)}+\varepsilon}}\nabla\mathrm{E}(\boldsymbol{w}^{(t)})_i
$$

学習率の割り引き方が、2乗和のルートから勾配の2乗の指数移動平均のルートに変化している点にも注意してください。

## NAG
現在のパラメータよりも一歩先のパラメータを仮定することでパラメータの変化量をより適切な方向に整える


$$
\boldsymbol{w}_t\leftarrow\beta_1\boldsymbol{m}_(t-1)+\eta\Delta(\boldsymbol{w}_t+\beta_1\boldsymbol{m}_t)
$$

$$
\boldsymbol{w}_{t+1}\leftarrow\boldsymbol{w}_t-\boldsymbol{m}_t
$$


## AdaDelta

RMSpropによって学習率が不可逆的に悪化することを防ぐことができましたが、AdaGradの全体の学習率に鋭敏であるという性質はそのままです。

この全体の学習率への鋭敏性、つまり問題設定毎に適切な学習率が変化してしまうという問題は、

実は更新量と勾配の次元の不一致を学習率で調整していることによるものです。（ここでの次元は物理的な次元のことで、いわゆる単位に相当するものです）

そこで、AdaDeltaではそうした次元の不一致を加味して自動的に適切な学習率が設定されるようにしています。

具体的には、勾配の2乗の指数移動平均に加えて、更新量の2乗の指数移動平均をもちい、両者の比を学習率として設定しています。

（なぜこれで次元の不一致に対処可能かは詳しく扱いませんが、Newton法が次元に対してロバストである＋Hessian逆行列の近似を利用して導出されます）

Kerasでは、`keras.optimizers.Adadelta`クラスを用います。RMSpropと同様に、更新量と勾配の指数移動平均を制御するパラメータ`rho`を設定できます。

通常`rho=0.95`とすることが推奨されています。

なお、Kerasの実装では一応学習率`lr`を設定できるようになっていますが、AdaDeltaの提案論文では学習率は自動的に決定されるものとしている上、

Kerasの公式HPでも`lr`はデフォルトのままとすることを推奨しているため、学習率の設定は基本的に不要です。

```py
# Adadeltaの実装例
from keras.optimizers import Adadelta
model.compile(loss='categorical_crossentropy', optimizer=Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0))
```

<small>
<参考>

更新式は次の通りです。（$\Delta\boldsymbol{w}^{(t)}_i$、$\nabla\mathrm{E}(\boldsymbol{w}^{(t)})_i$：AdaGradと同じ、$\rho$：勾配・更新量情報の減衰率（引数rho））

$$
    u_i^{(t)} = \rho u_i^{(t-1)} + (1-\rho)(\Delta\boldsymbol{w}^{(t)}_i)^2 \quad (u_i^{(0)}=0)\\
    v_i^{(t)} = \rho v_i^{(t-1)} + (1-\rho)(\nabla\mathrm{E}(\boldsymbol{w}^{(t)})_i)^2 \quad (v_i^{(0)}=0)\\
    \Delta\boldsymbol{w}^{(t)}_i=-\frac{\sqrt{u_i^{(t)}+\varepsilon}}{\sqrt{v_i^{(t)}+\varepsilon}}\nabla\mathrm{E}(\boldsymbol{w}^{(t)})_i
$$
</small>

## Adam

AdaDeltaとは異なるRMSpropの改良法としてAdamが挙げられます。

Adamでは、各方向への勾配の2乗に加えて勾配自身も、指数移動平均による推定値に置き換えています。

これにより、ある種Momentumと似た効果が期待できます。

Kerasでは、`keras.optimizers.Adam`クラスを使用します。

パラメータとしては、勾配、勾配の2乗それぞれの指数移動平均を制御するパラメータとして`beta_1,beta_2`が新たに指定可能です。

といっても、ほとんどの場合はデフォルトのパラメータが推奨され、実際に使用されています。

```py
# Adamの実装例
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
)
```

<small>
<参考>

更新式は次の通りです。（$\Delta\boldsymbol{w}^{(t)}_i$、$\nabla\mathrm{E}(\boldsymbol{w}^{(t)})_i$、$\eta$：RMSpropと同じ、$\rho_1,\rho_2$：勾配情報の減衰率（引数beta_1,beta_2））

$$
    m_i^{(t)} = \rho_1 m_i^{(t-1)} + (1-\rho_1)\nabla\mathrm{E}(\boldsymbol{w}^{(t)})_i \quad (m_i^{(0)}=0)\\
    v_i^{(t)} = \rho_2 v_i^{(t-1)} + (1-\rho_2)(\nabla\mathrm{E}(\boldsymbol{w}^{(t)})_i)^2 \quad (v_i^{(0)}=0)\\
    \hat{m}_i^{(t)} = \frac{m_i^{(t)}}{1-(\rho_1)^t}\\
    \hat{v}_i^{(t)} = \frac{v_i^{(t)}}{1-(\rho_2)^t}\\
    \Delta\boldsymbol{w}^{(t)}_i=-\frac{\eta}{\sqrt{\hat{v}_i^{(t)}+\varepsilon}}\hat{m}_i^{(t)}
$$

なお、$m_i^{(t)},v_i^{(t)}$を$\hat{m}_i^{(t)},\hat{v}_i^{(t)}$に変換しているのは、

各々勾配の1次モーメントと2次モーメントの推定量としてみた時、バイアスを持ってしまうため、その分を補正する役割です。
</small>